In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')

from matplotlib.pyplot import imshow
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
import os,sys

Using TensorFlow backend.


In [3]:
cd E:\dog-breed-classifier

E:\dog-breed-classifier


In [6]:
def load_array(fname):
    return np.load(open(fname,'rb'))
def load_Cropped_data():
    X_train=load_array('train_dataset.npy')
    train_labels=load_array('train_labels.npy')
    X_test=load_array('test_dataset.npy','rb')
    test_labels=load_array('test_labels.npy')
    return X_train,train_labels,X_test,test_labels

In [7]:
train_target=load_array('train_labels.npy')
valid_target=load_array('valid_labels.npy')

In [17]:
train_tensor=load_array('train_tensor.npy')
valid_tensor=load_array('valid_tensor.npy')

---
<a id='step3'></a>
# Create a CNN to Classify Dog Breeds (from Scratch)
Here we are setting Up an architecture to classify Dog Breeds.
The main Intitution Behind this is VGG16 network.


### Pre-process the Data

We rescale the images by dividing every pixel in every image by 255.

In [18]:
def Normalize_Input(X):
    minimum=0
    maximum=255
    X-minimum/(maximum-minimum)
    return X                

# pre-process the data for Keras
train_tensor=Normalize_Input(train_tensor)
valid_tensor=Normalize_Input(valid_tensor)

## Image Augmentation

In [13]:
from keras.preprocessing.image import ImageDataGenerator

# create and configure augmented image generator
datagen = ImageDataGenerator(
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True) # randomly flip images horizontally

# fit augmented image generator on data
datagen.fit(train_tensors)


## Define my own architecture

In [14]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(BatchNormalization(input_shape=(224, 224, 3)))
model.add(Conv2D(filters=16, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(GlobalAveragePooling2D())

model.add(Dense(133, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 111, 111, 16)      64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 54, 54, 32)        128       
__________

### Compile the Model

In [124]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the Model

We train our model in the code cell below.  Notice that I am using model checkpointing to save the model that attains the best validation loss.

In [129]:
epochs = 10
batch_size = 20

checkpointer = ModelCheckpoint(filepath='saved_models/weights.bestaugmented.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

### Using Image Augmentation
model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=batch_size),
                    validation_data=(valid_tensors, valid_targets), 
                    steps_per_epoch=train_tensors.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/10
334/334 [==============================] - 62s - loss: 4.5257 - acc: 0.0458 - val_loss: 4.3222 - val_acc: 0.0599
Epoch 2/10
334/334 [==============================] - 58s - loss: 4.0772 - acc: 0.0862 - val_loss: 4.0119 - val_acc: 0.0886
Epoch 3/10
334/334 [==============================] - 58s - loss: 3.7160 - acc: 0.1383 - val_loss: 3.6913 - val_acc: 0.1198
Epoch 4/10
334/334 [==============================] - 58s - loss: 3.4092 - acc: 0.1834 - val_loss: 3.4037 - val_acc: 0.1677
Epoch 5/10
334/334 [==============================] - 58s - loss: 3.1330 - acc: 0.2362 - val_loss: 3.1825 - val_acc: 0.2240
Epoch 6/10
334/334 [==============================] - 59s - loss: 2.8870 - acc: 0.2856 - val_loss: 3.0967 - val_acc: 0.2299
Epoch 7/10
334/334 [==============================] - 58s - loss: 2.6624 - acc: 0.3249 - val_loss: 2.9826 - val_acc: 0.2623
Epoch 8/10
334/334 [==============================] - 57s - loss: 2.4734 - acc: 0.3635 - val_loss: 2.7162 - val_acc: 0.3341
Epoch 9/

### Load the Model with the Best Validation Loss

In [137]:
model.load_weights('saved_models/weights.bestaugmented.from_scratch.hdf5')

### Test the Model

Let's try out our model on the test dataset of dog images. This will be our "test accuracy". 

In [131]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 33.6124%


### More Training
We can resume training to see if accuracy improves

In [134]:
batch_size = 20
epochs = 5

model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=batch_size),
                    validation_data=(valid_tensors, valid_targets), 
                    steps_per_epoch=train_tensors.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/5
334/334 [==============================] - 59s - loss: 1.8750 - acc: 0.4978 - val_loss: 2.5731 - val_acc: 0.3760
Epoch 2/5
334/334 [==============================] - 58s - loss: 1.7306 - acc: 0.5391 - val_loss: 2.2637 - val_acc: 0.4096
Epoch 3/5
334/334 [==============================] - 58s - loss: 1.6036 - acc: 0.5707 - val_loss: 2.2063 - val_acc: 0.4240
Epoch 4/5
334/334 [==============================] - 58s - loss: 1.4755 - acc: 0.5946 - val_loss: 2.1927 - val_acc: 0.4347
Epoch 5/5
334/334 [==============================] - 58s - loss: 1.3494 - acc: 0.6349 - val_loss: 2.1417 - val_acc: 0.4455


In [136]:
batch_size = 20
epochs = 5

model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=batch_size),
                    validation_data=(valid_tensors, valid_targets), 
                    steps_per_epoch=train_tensors.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/5
334/334 [==============================] - 59s - loss: 1.2430 - acc: 0.6597 - val_loss: 2.0425 - val_acc: 0.4611
Epoch 2/5
334/334 [==============================] - 58s - loss: 1.1431 - acc: 0.6880 - val_loss: 2.0567 - val_acc: 0.4359
Epoch 3/5
334/334 [==============================] - 58s - loss: 1.0465 - acc: 0.7112 - val_loss: 2.0488 - val_acc: 0.4874
Epoch 4/5
334/334 [==============================] - 57s - loss: 0.9818 - acc: 0.7304 - val_loss: 1.9496 - val_acc: 0.4778
Epoch 5/5
334/334 [==============================] - 58s - loss: 0.9059 - acc: 0.7425 - val_loss: 2.0029 - val_acc: 0.4802


In [139]:
batch_size = 64
epochs = 5

model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=batch_size),
                    validation_data=(valid_tensors, valid_targets), 
                    steps_per_epoch=train_tensors.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/5
104/104 [==============================] - 58s - loss: 0.6297 - acc: 0.8379 - val_loss: 1.7295 - val_acc: 0.5281
Epoch 2/5
104/104 [==============================] - 55s - loss: 0.5352 - acc: 0.8737 - val_loss: 1.7202 - val_acc: 0.5305
Epoch 3/5
104/104 [==============================] - 55s - loss: 0.4993 - acc: 0.8860 - val_loss: 1.7535 - val_acc: 0.5257
Epoch 4/5
104/104 [==============================] - 56s - loss: 0.4622 - acc: 0.8931 - val_loss: 1.7339 - val_acc: 0.5150
Epoch 5/5
104/104 [==============================] - 54s - loss: 0.4275 - acc: 0.9047 - val_loss: 1.7621 - val_acc: 0.5174


## Test the Model Again
After we test the model again we see that we improved test accuracy up to 51%

In [140]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 51.0766%
